In [1]:
import numpy as np
import numpy.linalg as la


##### $f(\mathbf x)=\left(x_{1} - 2\right)^{4} + \left(x_{1} - x_{2}\right)^{2}$


In [2]:
f1 = lambda x: 4 * (x[0] - 2)**4 + 2 * (x[0] - 2 * x[1])**2
df1 = lambda x: np.array([4 * x[0] - 8 * x[1] + 16 * (x[0] - 2)**3, -8 * x[0] + 16 * x[1]])
ddf1 = lambda x: np.array([[12 * (x[0] - 2)**2 + 2, -2], [-2, 2]])

x0 = np.array([1, 1.0])
Q0 = np.array([[14., -2.],
               [-2., 2.]])


### Armijo's inequality
$$f(\mathbf {x}^k + \alpha \mathbf {d}^k) \le f(\mathbf {x}^k) + \sigma\alpha <\nabla f(\mathbf {x}^k), \mathbf {d}^k>\\$$

In [3]:
def step_armijo(f, d, x, MAX_ITER=1000):
    # return step size alpha of armijo rule
    alpha = 2  # initial, step size
    theta = 0.5
    sigma = 0.1
    for _ in range(MAX_ITER):
        if f(x + alpha * d) <= f(x) + sigma * alpha * np.dot(-d, d):
            return alpha
        alpha *= theta


### QuasiNewton

##### Davidon-Fletcher-Powell Method
$$
Q_{k+1} = Q_k + \frac{r_k r_k^T}{r_k, s_k} - \frac{Q_ks_k {(Q_ks_k)}^T}{{(Q_ks_k)}^Ts_k}
$$

♦

In [4]:
def quasi_newton_DFP(f, grad_f, x0, Q, eps=1e-3, MAX_ITER=1000):
    for i in range(MAX_ITER):
        # line search
        d = -Q @ grad_f(x0)
        alpha = step_armijo(f, d, x0)
        # update x
        x_new = x0 + alpha * d
        error = la.norm(d)
        print(f"{i:02d}, {x_new}, {error:0.12f}")
        if error < eps:
            break
        # common operations
        r = x_new - x0
        s = grad_f(x_new) - grad_f(x0)
        Qs = Q @ s
        # update for next step
        Q = Q + np.outer(r, r) / np.dot(r, s) - np.outer(Qs, Qs) / np.dot(Qs, s)
        x0 = x_new.copy()


In [5]:
print("Davidon-Fletcher-Powell Method")
print("k\t\tx[k]\t\terror")
quasi_newton_DFP(f1, df1, x0, Q0);


Davidon-Fletcher-Powell Method
k		x[k]		error
00, [1. 1.], 301.250726140204
01, [0.94679638 0.5245283 ], 7.655025400866
02, [1.79215001 0.80501583], 0.445335831130
03, [1.80467621 0.98875323], 0.092081943225
04, [1.80459098 0.90282218], 0.085931094859
05, [1.81547888 0.90722632], 0.005872455806
06, [1.92683742 0.95863168], 0.061325431526
07, [1.93814576 0.96903905], 0.007684267335
08, [1.94950607 0.97554251], 0.006545068948
09, [1.96442107 0.98301902], 0.008341993436
10, [1.96837713 0.98484185], 0.008711632561
11, [1.96837713 0.98484185], 0.011610004829
12, [1.97217833 0.98638961], 0.008208460257
13, [1.97217833 0.98638961], 0.006710489306
14, [1.97217833 0.98638961], 0.007743456268
15, [1.97217833 0.98638961], 0.006441705260
16, [1.97217833 0.98638961], 0.007563378841
17, [1.97217833 0.98638961], 0.006523135562
18, [1.97217833 0.98638961], 0.007621618967
19, [1.97217833 0.98638961], 0.006572304945
20, [1.97217833 0.98638961], 0.007679798460
21, [1.97217833 0.98638961], 0.006686719408


##### Broyden-Fletcher-Goldfarb-Shanno Method 

$$
Q_{k+1} = Q_k + \frac{(r_k  - Q_ks_k)r_k^T + r_k{(r_k  - Q_ks_k)}^T}{r_k^Ts_k} - \frac{{(r_k  - Q_ks_k)}^Ts_k}{{(r_k^Ts_k)}^2}r_kr_k^T
$$

In [6]:
def quasi_newton_BFGS(f, grad_f, x0, Q, eps=1e-3, MAX_ITER=1000):
    for i in range(MAX_ITER):
        # line search
        d = -Q @ grad_f(x0)
        alpha = step_armijo(f, d, x0)
        # update x
        x_new = x0 + alpha * d
        error = la.norm(d)
        print(f"{i:02d}, {x_new}, {error:0.12f}")
        if error < eps:
            break
        # common operations
        r = x_new - x0
        s = grad_f(x_new) - grad_f(x0)
        r_Qs = r - Q @ s
        rs = np.dot(r, s)
        outer = np.outer(r_Qs, r)
        # update for next step
        Q = Q + (outer + outer.T) / rs - np.dot(r_Qs, s) * np.outer(r, r) / rs**2
        x0 = x_new.copy()


In [7]:
print("Broyden-Fletcher-Goldfarb-Shanno Method")
print("k\t\tx[k]\t\terror")
quasi_newton_BFGS(f1, df1, x0, Q0)


Broyden-Fletcher-Goldfarb-Shanno Method
k		x[k]		error
00, [1. 1.], 301.250726140204
01, [0.92881944 0.41666667], 9.402562364183
02, [1.9597222  0.91586226], 0.572703399195
03, [1.95979204 0.98103256], 0.065170343153
04, [1.95984535 0.97992275], 0.001111094640
05, [1.95995456 0.97997116], 0.000059726586


#### Examen Final
##### 3. Usando los métodos de Newton y Cuasi Newton aproxime la solución del siguiente problema, use $x_0 = [0, 2.3]$.

\begin{align*}
    \min&\quad - 2 e^{-\left(x - 1\right)^{2} - \left(y - 1\right)^{2}}\\
    \text{s.t.}&\quad x\in \mathbb R^2
\end{align*}

In [70]:
f2 = lambda x: -2 * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2)
df2 = lambda x: np.array([-2 * (x[0] - 1) * f2(x),
                          -2 * (x[1] - 1) * f2(x)])
ddf2 = lambda x: np.array([[-2 * f2(x) * (1 - 2 * (x[0] - 1)**2), 4 * (x[0] - 1) * (x[1] - 1) * f2(x)],
                           [4 * (x[0] - 1) * (x[1] - 1) * f2(x), -2 * f2(x) * (1 - 2 * (x[1] - 1)**2)]])
x0 = np.array([0.0, 2.3])
Q0 = np.array([[1, 0.0],
               [0, 2.0]])


In [71]:
print("Davidon-Fletcher-Powell Method")
print("k\t\tx[k]\t\terror")
quasi_newton_DFP(f2, df2, x0, Q0)


Davidon-Fletcher-Powell Method
k		x[k]		error
00, [0.54304751 0.88807646], 0.756377664152
01, [0.55552338 0.86081751], 1.918610079027
02, [0.70804655 0.79214342], 0.083635292810
03, [1.17250465 0.70324337], 0.236444783065
04, [0.74420789 0.87230328], 0.230227800554
05, [1.13309972 0.7981925 ], 0.197945233707
06, [0.81011709 0.92449781], 0.173400412442
07, [1.11011972 0.86472349], 0.152949786972
08, [0.85361328 0.9610643 ], 0.137001010367
09, [1.09814195 0.90784437], 0.125126568089
10, [0.87807495 0.9855175 ], 0.116686120215
11, [1.09385669 0.9339416 ], 0.110929970492
12, [0.89048036 1.00140607], 0.107137045343
13, [0.99189357 0.97536677], 0.104702838877
14, [0.99815125 0.98557142], 0.005985265583
15, [1.00418444 1.01302904], 0.014056315877
16, [1.00026367 0.99994189], 0.013661840670
17, [0.99984367 1.00002594], 0.000214165877


In [72]:
print("Broyden-Fletcher-Goldfarb-Shanno Method")
print("k\t\tx[k]\t\terror")
quasi_newton_BFGS(f2, df2, x0, Q0)


Broyden-Fletcher-Goldfarb-Shanno Method
k		x[k]		error
00, [0.54304751 0.88807646], 0.756377664152
01, [0.54304751 0.88807646], 18.522433883847
02, [0.86598678 1.03226337], 5.658655817181
03, [1.04577316 0.99680826], 0.183249034296
04, [0.99924462 0.9991517 ], 0.046587514298
05, [1.00000923 1.00015473], 0.001261231028
06, [1.00000395 0.99999699], 0.000157833869
